In this file we load index fund data that we got by scrapping the web. We load them, transform to necessary format, get sectoral data from it.

In [2]:
import pandas as pd

Load the data

In [154]:
# Specify the path to your Excel file
index_file_path = './Data/Index_Data/Prepared_Index_Data/20241905_Index_fund_data_v2.xlsx'

index_df = pd.read_excel(index_file_path)
index_df.head()


,Sr No,Company,Market Cap,Index Fund,Company Name,Industry,Symbol,Series
0,1.0,360 ONE,27788.56,NIFTY 500,360 ONE WAM Ltd.,Financial Services,360ONE,EQ
1,3.0,360 ONE,27779.59,NIFTY MIDSML 400,360 ONE WAM Ltd.,Financial Services,360ONE,EQ
2,2.0,360 ONE,27779.59,NIFTY SMLCAP 250,360 ONE WAM Ltd.,Financial Services,360ONE,EQ
3,4.0,360 ONE,27779.59,NIFTY TOTAL MKT,360 ONE WAM Ltd.,Financial Services,360ONE,EQ
4,5.0,3M India,34043.12,NIFTY 500,3M India Ltd.,Diversified,3MINDIA,EQ


We load the mapping files. These files were manually prepared. Stock_name_map contains stock id and name in mutual fund data that we have and index fund data that we have. Stock_sec_map is stock_id to sector_id mapping and finally index_fund_map is index fund id to its name map and it also contains last year's return (acting as proxy of expected return) 

In [155]:
stock_name_map      = pd.read_csv('./Data/MF_Data/Results/End_Product/stock_name_map.csv')
stock_sec_map       = pd.read_csv('./Data/MF_Data/Results/End_Product/stock_sector_mapping.csv')
index_fund_map      = pd.read_csv('./Data/Index_Data/Prepared_Index_Data/mapping/index_fund_map.csv')
index_fund_map.head()

,INDEX_FUND,FUND_ID,EXP_RET
0,NIFTY 100,ID1,0.514741
1,NIFTY 200,ID2,0.580542
2,NIFTY 50,ID3,0.325239
3,NIFTY 500,ID4,0.630966
4,NIFTY ALPHA 50,ID5,1.590848


In [1]:
index_fund_map

NameError: name 'index_fund_map' is not defined

We merge these map data with Index fund data frame that we have

In [156]:
stock_name_map_tmp    = stock_name_map[['Index_fund_Name','Stock_ID']]
stock_sec_map_tmp     = stock_sec_map[['Stock_ID','Sector_ID']] 
index_fund_map_tmp    = index_fund_map[['INDEX_FUND','FUND_ID']]

index_df              = pd.merge(index_df,stock_name_map_tmp,left_on="Company Name", right_on = "Index_fund_Name", how = "left")
index_df              = pd.merge(index_df,stock_sec_map_tmp,on="Stock_ID", how = "left")
index_df              = pd.merge(index_df,index_fund_map_tmp,left_on="Index Fund",right_on="INDEX_FUND", how = "left")

index_df.head()

,Sr No,Company,Market Cap,Index Fund,Company Name,Industry,Symbol,Series,Index_fund_Name,Stock_ID,Sector_ID,INDEX_FUND,FUND_ID
0,1.0,360 ONE,27788.56,NIFTY 500,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,360 ONE WAM Ltd.,S1,SEC1,NIFTY 500,ID4
1,3.0,360 ONE,27779.59,NIFTY MIDSML 400,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,360 ONE WAM Ltd.,S1,SEC1,NIFTY MIDSML 400,ID24
2,2.0,360 ONE,27779.59,NIFTY SMLCAP 250,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,360 ONE WAM Ltd.,S1,SEC1,NIFTY SMLCAP 250,ID33
3,4.0,360 ONE,27779.59,NIFTY TOTAL MKT,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,360 ONE WAM Ltd.,S1,SEC1,NIFTY TOTAL MKT,ID35
4,5.0,3M India,34043.12,NIFTY 500,3M India Ltd.,Diversified,3MINDIA,EQ,3M India Ltd.,S3,SEC3,NIFTY 500,ID4


Calculate stock wise weight in the index fund data

In [157]:
index_df_tmp         = index_df.groupby('INDEX_FUND')['Market Cap'].sum()
index_df_tmp         = index_df_tmp.reset_index()
index_df_tmp         = index_df_tmp.rename(columns = {'INDEX_FUND':'INDEX_FUND','Market Cap':'TOTAL_MCAP'})

index_df              = pd.merge(index_df,index_df_tmp,left_on="INDEX_FUND",right_on="INDEX_FUND", how = "left")


index_df['WEIGHT']    = index_df['Market Cap']/index_df['TOTAL_MCAP'] 

index_df          = index_df[['FUND_ID','Stock_ID','Sector_ID','WEIGHT']]
index_df          = index_df.rename(columns={'Stock_ID':'STOCK_ID','Sector_ID':'SECTOR_ID' })
index_df.head()

,FUND_ID,STOCK_ID,SECTOR_ID,WEIGHT
0,ID4,S1,SEC1,0.000748
1,ID24,S1,SEC1,0.002419
2,ID33,S1,SEC1,0.006831
3,ID35,S1,SEC1,0.000720
4,ID4,S3,SEC3,0.000917


In [159]:
index_df        = index_df.drop_duplicates(subset=['FUND_ID','STOCK_ID'], keep = 'last')

Preparing data at granularity of sector

In [149]:
index_df_sec    = index_df.groupby(['SECTOR_ID','FUND_ID'])['WEIGHT'].sum() 
index_df_sec    = index_df_sec.reset_index()
index_df_sec    = index_df_sec.sort_values(by = 'FUND_ID')
#index_df_sec[index_df_sec['FUND_ID']=='ID1']['WEIGHT'].sum()
index_df_sec.head()

,SECTOR_ID,FUND_ID,WEIGHT
196,SEC127,ID1,0.013866
919,SEC46,ID1,0.017030
1242,SEC77,ID1,0.006714
277,SEC137,ID1,0.006126
142,SEC12,ID1,0.013796


In [151]:
index_df_sec.to_csv('./Data/Index_Data/Prepared_Index_Data/index_fund_sector_weights.csv', index=False)

In [121]:
# Perform the pivot operation
index_df = index_df.pivot(index=['STOCK_ID'], columns='FUND_ID', values='WEIGHT').fillna(0)
index_df.reset_index(inplace=True)
index_df.columns.name = None

# Display the resulting DataFrame
print(index_df)

# Optionally, save the pivot table to a CSV file
index_df.to_csv('./Data/Index_Data/Prepared_Index_Data/index_fund_pivoted_weights.csv',index=False)


    STOCK_ID       ID1      ID10  ID11  ID12      ID13  ID14      ID15  ID16  \
0         A1  0.000000  0.000000   0.0   0.0  0.009632   0.0  0.002124   0.0   
1         A2  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
2         N1  0.004605  0.028019   0.0   0.0  0.000000   0.0  0.003573   0.0   
3        N10  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
4       N100  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
..       ...       ...       ...   ...   ...       ...   ...       ...   ...   
747     S991  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
748     S992  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
749     S993  0.003535  0.000000   0.0   0.0  0.012429   0.0  0.002743   0.0   
750     S996  0.003744  0.000000   0.0   0.0  0.000000   0.0  0.002907   0.0   
751     S999  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   

     ID17  ...  ID34      ID35      ID3